# Random Forest Forecast

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score
from pandas import DataFrame
import pickle
import seaborn as sns

# Load and preprocess the data
data_file = "mulu/mulu-rainfall-daily"
df = pd.read_csv(f'{data_file}.csv', encoding='latin')
df["DateTime"] = pd.to_datetime(df["DateTime"],format="%Y-%m-%d")
df.set_index("DateTime",inplace=True)
# df['Wind Speed'] = df['Wind Speed'].replace(np.nan, 0)
# df = df.fillna(method='ffill')

Feature Engineering

In [ ]:
# Define the sets of features for interaction
source1 = ['Rainfall']
source2 = ['TOTAL', 'ClimAdjust', 'ANOM']
source3 = ['Temperature', 'DewPoint', 'Humidity', 'WindSpeed', 'Pressure']

# Create interaction features
for s1 in source1:
    for s2 in source2:
        # Interaction between source1 and source2
        interaction_term = f'{s1}_{s2}'
        df[interaction_term] = df[s1] * df[s2]
        
    for s3 in source3:
        # Interaction between source1 and source3
        interaction_term = f'{s1}_{s3}'
        df[interaction_term] = df[s1] * df[s3]
        
for s2 in source2:
    for s3 in source3:
        # Interaction between source2 and source3
        interaction_term = f'{s2}_{s3}'
        df[interaction_term] = df[s2] * df[s3]

for i in range(len(source3)):
    for j in range(i + 1, len(source3)):  # Avoid duplicate pairs and self-interaction
        s3_1 = source3[i]
        s3_2 = source3[j]
        interaction_term = f'{s3_1}_{s3_2}'
        df[interaction_term] = df[s3_1] * df[s3_2]
        
print(df.columns.to_list())

In [ ]:
# Create rolling statistics for 24, 48, 72 hours (1, 2, 3 days)
for window in [7,14,30]:
    df[f"Rainfall_{window}d_mean"] = df["Rainfall"].rolling(window).mean()
    df[f"Rainfall_{window}d_std"] = df["Rainfall"].rolling(window).std()
    df[f"Rainfall_{window}d_sum"] = df["Rainfall"].rolling(window).sum()
    df[f"Rainfall_{window}d_min"] = df["Rainfall"].rolling(window).min()
    df[f"Rainfall_{window}d_max"] = df["Rainfall"].rolling(window).max()
    df[f"Rainfall_{window}d_median"] = df["Rainfall"].rolling(window).median()

In [ ]:
for lag in range(1, 91):
    df[f'Rainfall_lag_{lag}'] = df['Rainfall'].shift(lag)
    df[f'Temperature_lag_{lag}'] = df['Rainfall'].shift(lag)

In [ ]:
df.dropna(inplace=True)

In [ ]:
corr_matrix = df.iloc[:, :43].corr(numeric_only=True)
print(corr_matrix)

plt.figure(figsize=(80,40))
sns.heatmap(corr_matrix, cmap='coolwarm', annot=True)

Dimensionality Reduction

In [ ]:
df_dim_red = df.drop(columns=['TOTAL', 'ClimAdjust', 'ANOM', 'DewPoint', 'WindSpeed', 'Pressure', 'TOTAL_DewPoint', 'TOTAL_WindSpeed', 'TOTAL_Pressure', 'ClimAdjust_DewPoint', 'ClimAdjust_WindSpeed', 'ClimAdjust_Pressure', 'ANOM_Temperature', 'ANOM_DewPoint', 'ANOM_Humidity', 'ANOM_WindSpeed', 'ANOM_Pressure', 'Temperature_WindSpeed', 'DewPoint_WindSpeed', 'DewPoint_Pressure', 'Humidity_WindSpeed', 'WindSpeed_Pressure', 'Rainfall_14d_min', 'Rainfall_30d_min'])
df_dim_red = df_dim_red.select_dtypes(include=['number'])

In [ ]:
corr_matrix = df_dim_red.corr(numeric_only=True)
print(corr_matrix)

plt.figure(figsize=(80,40))
sns.heatmap(corr_matrix, cmap='coolwarm', annot=True)

In [ ]:
# Scaling the features and target separately
feature_scaler = MinMaxScaler()
target_scaler = MinMaxScaler()

# Separate features (lags of all columns) and target (Current_Rainfall)
features = df_dim_red.drop(columns=['Rainfall']).reset_index(drop=True)
target = df_dim_red['Rainfall']

# Scale features and target
features_scaled = feature_scaler.fit_transform(features)
target_scaled = target_scaler.fit_transform(target.values.reshape(-1, 1))

# RF_model.pkl - Rainfall lagged for 90 days
# RF_model1.pkl - Rainfall lagged for 30 days
with open('RF_model3.pkl', 'rb') as f:
    rf = pickle.load(f)

In [ ]:
# Predict on the test set
y_pred_scaled = rf.predict(features_scaled)
y_pred = target_scaler.inverse_transform(y_pred_scaled.reshape(-1, 1)).ravel()  # Convert predictions back to original scale
y_test_original = target_scaler.inverse_transform(target_scaled).ravel()  # Convert test targets back to original scale

test_result = pd.DataFrame({
    'y_pred': y_pred,
    'y_test_original': y_test_original
})

# Calculate evaluation metrics
mae = mean_absolute_error(y_test_original, y_pred)
mse = mean_squared_error(y_test_original, y_pred)
rmse = np.sqrt(mse)
r2 = r2_score(y_test_original, y_pred)

# Print evaluation metrics
print("Evaluation Metrics:")
print(f"Mean Absolute Error (MAE): {mae:.2f}")
print(f"Mean Squared Error (MSE): {mse:.2f}")
print(f"Root Mean Squared Error (RMSE): {rmse:.2f}")
print(f"R² Score: {r2:.2f}")

plt.figure(figsize=(10, 6))
plt.plot(y_test_original, label='Actual Rainfall', color='blue', linestyle='-', linewidth=2)
plt.plot(y_pred, label='Predicted Rainfall', color='orange', alpha=0.6, linestyle='--', linewidth=2)
plt.xlabel('Time')
plt.ylabel('Rainfall (mm)')
plt.title(f'Actual vs Predicted Rainfall - {data_file}')
plt.legend()
plt.grid(True)
plt.tight_layout()
# plt.xlim(30,40)
plt.show()

In [ ]:
test_result

Forecasting 30 days

In [ ]:
# full code of forecasting function

import pandas as pd
import numpy as np
import pickle
from sklearn.preprocessing import MinMaxScaler

def forecast_30_days_ahead(data_file, model_file):
    # Load and preprocess the data
    df = pd.read_csv(f'{data_file}.csv', encoding='latin')
    df["DateTime"] = pd.to_datetime(df["DateTime"], format="%Y-%m-%d")
    df.set_index("DateTime", inplace=True)

    # Define the sets of features for interaction
    source1 = ['Rainfall']
    source2 = ['TOTAL', 'ClimAdjust', 'ANOM']
    source3 = ['Temperature', 'DewPoint', 'Humidity', 'WindSpeed', 'Pressure']

    # Create interaction features
    for s1 in source1:
        for s2 in source2:
            interaction_term = f'{s1}_{s2}'
            df[interaction_term] = df[s1] * df[s2]
        for s3 in source3:
            interaction_term = f'{s1}_{s3}'
            df[interaction_term] = df[s1] * df[s3]
    for s2 in source2:
        for s3 in source3:
            interaction_term = f'{s2}_{s3}'
            df[interaction_term] = df[s2] * df[s3]
    for i in range(len(source3)):
        for j in range(i + 1, len(source3)):
            s3_1 = source3[i]
            s3_2 = source3[j]
            interaction_term = f'{s3_1}_{s3_2}'
            df[interaction_term] = df[s3_1] * df[s3_2]

    # Create rolling statistics for 7, 14, 30 days
    for window in [7, 14, 30]:
        df[f"Rainfall_{window}d_mean"] = df["Rainfall"].rolling(window).mean()
        df[f"Rainfall_{window}d_std"] = df["Rainfall"].rolling(window).std()
        df[f"Rainfall_{window}d_sum"] = df["Rainfall"].rolling(window).sum()
        df[f"Rainfall_{window}d_min"] = df["Rainfall"].rolling(window).min()
        df[f"Rainfall_{window}d_max"] = df["Rainfall"].rolling(window).max()
        df[f"Rainfall_{window}d_median"] = df["Rainfall"].rolling(window).median()

    # Create lag features
    for lag in range(1, 91):
        df[f'Rainfall_lag_{lag}'] = df['Rainfall'].shift(lag)
        df[f'Temperature_lag_{lag}'] = df['Temperature'].shift(lag)

    # Drop rows with NaN values
    df.dropna(inplace=True)

    # Select relevant columns for dimensionality reduction
    columns_to_drop = ['TOTAL', 'ClimAdjust', 'ANOM', 'DewPoint', 'WindSpeed', 'Pressure',
                       'TOTAL_DewPoint', 'TOTAL_WindSpeed', 'TOTAL_Pressure',
                       'ClimAdjust_DewPoint', 'ClimAdjust_WindSpeed', 'ClimAdjust_Pressure',
                       'ANOM_Temperature', 'ANOM_DewPoint', 'ANOM_Humidity',
                       'ANOM_WindSpeed', 'ANOM_Pressure', 'Temperature_WindSpeed',
                       'DewPoint_WindSpeed', 'DewPoint_Pressure', 'Humidity_WindSpeed',
                       'WindSpeed_Pressure', 'Rainfall_14d_min', 'Rainfall_30d_min']
    
    df_dim_red = df.drop(columns=columns_to_drop)
    df_dim_red = df_dim_red.select_dtypes(include=['number'])

    # Scaling the features and target separately
    feature_scaler = MinMaxScaler()
    target_scaler = MinMaxScaler()

    # Separate features (lags of all columns) and target (Current_Rainfall)
    features = df_dim_red.drop(columns=['Rainfall']).reset_index(drop=True)
    target = df_dim_red['Rainfall']

    # Scale features and target
    features_scaled = feature_scaler.fit_transform(features)
    target_scaled = target_scaler.fit_transform(target.values.reshape(-1, 1))

    # Load the pre-trained model
    with open(model_file, 'rb') as f:
        rf_model = pickle.load(f)

    # Forecast the next 30 days
    forecast_length = 30
    forecasted_values_scaled = rf_model.predict(features_scaled[-forecast_length:])
    
    # Convert predictions back to original scale
    forecasted_values = target_scaler.inverse_transform(forecasted_values_scaled.reshape(-1, 1)).ravel()

    return forecasted_values


data_file = "mulu/mulu-rainfall-daily"
model_file = "RF_model3.pkl"
forecasted_rainfall_30_days_ahead = forecast_30_days_ahead(data_file, model_file)
print(forecasted_rainfall_30_days_ahead)

[23.97378443  8.06467066 35.09062275 26.64443114 20.44394012  0.
 49.07972455  4.96287425  9.46047904 40.99023952  0.         11.01137725
  0.46526946 15.50898204  0.          0.          7.44431138  0.46526946
 11.47664671  0.46526946  2.40699401 29.03669162 13.71769461 17.0102515
  2.94670659 16.53412575 34.94949102  0.          0.          4.03233533]


C:\Users\User\AppData\Local\Temp\ipykernel_17092\776497792.py:50: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  df[f'Temperature_lag_{lag}'] = df['Temperature'].shift(lag)
C:\Users\User\AppData\Local\Temp\ipykernel_17092\776497792.py:49: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  df[f'Rainfall_lag_{lag}'] = df['Rainfall'].shift(lag)
C:\Users\User\AppData\Local\Temp\ipykernel_17092\776497792.py:50: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has po